# Finetuning LLaMa + Text-to-SQL 

This walkthrough shows you how to fine-tune LLaMa-7B on a Text-to-SQL dataset, and then use it for inference against
any database of structured data using LlamaIndex.

**NOTE**: This code is taken and adapted from Modal's `doppel-bot` repo: https://github.com/modal-labs/doppel-bot.
**NOTE**: A lot of the code is contained in the underlying Python scripts in the `src` directory. We definitely encourage you to go and take a look!

### Setup

NOTE: you will need to setup a Modal account + token in order to use this notebook.

In [6]:
! source env/bin/activate

In [2]:
from platform import python_version

print(python_version())

3.9.18


In [1]:
!pip install -r requirements.txt

  Using cached aiohttp-3.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached aiostream-0.5.2-py3-none-any.whl.metadata (9.9 kB)
  Using cached asgiref-3.7.2-py3-none-any.whl.metadata (9.2 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typer-0.9.0-py3-none-any.whl.metadata (14 kB)
  Using cached types_certifi-2021.10.8.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached types_toml-0.10.8.7-py3-none-any.whl.metadata (1.3 kB)
  U

In [4]:
!python3.9 -m pip install --upgrade modal

  Using cached modal-0.61.25-py3-none-any.whl.metadata (2.3 kB)
  Using cached grpclib-0.4.7-py3-none-any.whl
  Using cached synchronicity-0.6.2-py3-none-any.whl.metadata (7.8 kB)
Using cached modal-0.61.25-py3-none-any.whl (413 kB)
Using cached synchronicity-0.6.2-py3-none-any.whl (28 kB)
  Attempting uninstall: synchronicity
    Found existing installation: synchronicity 0.5.3
    Uninstalling synchronicity-0.5.3:
      Successfully uninstalled synchronicity-0.5.3
  Attempting uninstall: grpclib
    Found existing installation: grpclib 0.4.3
    Uninstalling grpclib-0.4.3:
      Successfully uninstalled grpclib-0.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
modal-client 0.50.3044 requires grpclib==0.4.3, but you have grpclib 0.4.7 which is incompatible.
modal-client 0.50.3044 requires synchronicity~=0.5.3, but you have synchronicity 0.6.2 which is inco

### Load Training Data for Finetuning LLaMa

We load data from `b-mc2/sql-create-context` on Hugging Face: https://huggingface.co/datasets/b-mc2/sql-create-context.

This dataset consists of tuples of natural language queries, create table statements, and ground-truth SQL queries. This is the dataset that we use to finetune our SQL model.

In [6]:
data_dir = "data_sql"

!modal run src.load_data_sql --data-dir {data_dir}

╭─ Modal Deprecation Warning (2024-03-01) ─────────────────────────────────────╮
│ Deprecated! Use `NetworkFileSystem.from_name(name, create_if_missing=True)`. │
│                                                                              │
│ Source: /home/dsl/Documents/fbk/modal_finetune_sql/src/common.py:57          │
│   output_vol = NetworkFileSystem.new(cloud="gcp").persisted("doppelbot-vol") │
╰──────────────────────────────────────────────────────────────────────────────╯
✓ Initialized. View run at 
https://modal.com/lattaruoloandrea/apps/ap-FLsZcfmS4zP1KwcyXYcOkD
⠋ Initializing...
⠸ Creating objects...objects...
└── ⠋ Creating mount /home/dsl/Documents/fbk/modal_finetune_sql/src: Uploaded 
⠦ Creating objects...
└── ⠸ Creating mount /home/dsl/Documents/fbk/modal_finetune_sql/src: Building 
⠏ Creating objects...
⠹ Creating objects...dsl/Documents/fbk/modal_finetune_sql/src
⠴ Creating objects...dsl/Documents/fbk/modal_finetune_sql/src
⠏ Creating objects...dsl/Documents/fbk/modal

In [10]:
load_data_sql()

ValueError: Invalid pattern: '**' can only be an entire path component

### Run Finetuning Script

We run our finetuning script on the loaded dataset.
The finetuning script contains the following components:
- We split the dataset into training and validation splits.
- We format each split into input/output tuples of token id's. This means that the labels are the same as inputs (loss signal is measured on full input, not just on the generated portion). 
- We use `LoraConfig` from `peft` for efficient fine-tuning.
- We use `transformers.Trainer` to actually run the training process.
- If a valid `WANDB_PROJECT` is specified, along with the relevant secret in Modal, then we will log results to wandb.

We use Modal to spin up an A100 to run our finetuning code. 

In [ ]:
data_dir = "data_sql"

!modal run src.finetune_sql --data-dir {data_dir}

### Evaluation

We provide a basic evaluation script over sample data from `sql-create-context` so that you can see for yourself how well the finetuned model performs vs. the baseline model.

In [17]:
!modal run src.eval_sql::main

✓ Initialized. View app at https://modal.com/apps/ap-JS10MTVAhat3jK4K8QwKyv
⠋ Initializing...
⠼ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠧ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠋ Creating objects...
├── ⠦ Creating download_models...
└── ⠦ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠸ Creating objects...
├── ⠏ Creating download_models...
└── ⠏ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠦ Creating objects...
├── ⠹ Creating download_models...
⠋ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠦ Creating download_models...
⠸ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠏ Creating download_models...
⠦ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── ⠹ Creating download_models...
⠏ Creating object

### Integrate Model with LlamaIndex

Now that the model is finetuned, the checkpoints and model binary are stored in a model directory (by default it is in `/vol/data_sql`).

We can now use this model in LlamaIndex for text-to-SQL applications.

Specifically, we provide an interface allowing users to define any `sqlite` data file, and then they can run queries over this data file. We first create and dump a sample `cities.db` file containing (city, population, country) tuples. We then run inference over this file.

#### Create sample db

In [9]:
# create sample 
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

In [10]:
db_file = "cities.db"
engine = create_engine(f"sqlite:///{db_file}")
metadata_obj = MetaData()

In [11]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [12]:
# insert sample rows
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000, "country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()

#### Run Inference

In [15]:
query = "Which city has the highest population?"

!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model True

✓ Initialized. View app at https://modal.com/apps/ap-BjYd5um2Efz70MftYxlyzE
⠋ Initializing...
⠹ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠴ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠏ Creating objects...
├── ⠦ Creating download_models...
⠹ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠴ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠇ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_sql/src
├── ⠋ Creating run_query...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠙ Creating objects...
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetu

In [16]:
# you can also choose to run the original (nonfinetuned model) to compare results
# note: it throws an error 

# use non-finetuned model
!modal run src.inference_sql_llamaindex::main --query '{query}' --sqlite-file-path {db_file} --model-dir "data_sql" --use-finetuned-model False

✓ Initialized. View app at https://modal.com/apps/ap-1n1QmIubsUDYdMnVNCTGzh
⠋ Initializing...
⠹ Creating objects...objects...
├── ⠋ Creating download_models...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠴ Creating objects...
├── ⠸ Creating download_models...
└── ⠸ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠏ Creating objects...
├── 🔨 Created download_models.
⠹ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠴ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
⠇ Creating objects...s/jerryliu/Programming/modal_finetune_sql/src
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_sql/src
├── ⠋ Creating run_query...
└── ⠋ Creating mount /Users/jerryliu/Programming/modal_finetune_sql/src: 
⠙ Creating objects...
├── 🔨 Created download_models.
├── 🔨 Created mount /Users/jerryliu/Programming/modal_finetune_

### (Optional) Download Model

If you want to download the model weights for your own use, just run the following script.

In [ ]:
from src.download_weights import main

main("out_model", model_dir="data_sql")